In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from dataclasses import dataclass

In [2]:
def matComp(xmin,xmax,ymin,ymax,dens):
    re=np.linspace(xmax,xmin,int(xmax-xmin)*dens)
    im=np.linspace(ymax,ymin,int(ymax-ymin)*dens)
    return re[np.newaxis,:]+im[:,np.newaxis]*1j

In [3]:
from math import log
@dataclass
class ConjMandelbrot:
    max_iter:int
    radio_esc:float
    def __cont__(self,c):
        return self.stability(c)==1
    def stability(self,c,smooth=False,clamp=True):
        value = self.esc_count(c,smooth)/self.max_iter
        return max(0.,min(value,1.)) if clamp else value
    def esc_count(self,c,smooth=False):
        z=0
        for i in range(self.max_iter):
            z=z**2+c
            if abs(z)>self.radio_esc:
                if smooth:
                    return i +1-log(log(abs(z)))/log(2)
                return i
        return self.max_iter

In [4]:
@dataclass
class ViewPort:
    image: Image.Image
    center:complex
    width:float
    @property
    def height(self):
        return self.scale*self.image.height
    @property
    def offset(self):
        return self.center+complex(-self.width, self.height)/2
    @property
    def scale(self):
        return self.width/self.image.width
    
    def __iter__(self):
        for y in range(self.image.height):
            for x in range(self.image.width):
                yield Pixel(self,x,y)


In [6]:
@dataclass
class Pixel:
    viewport:ViewPort
    x:int
    y:int
    
    @property
    def color(self):
        return self.viewport.image.getpixel((self.x,self.y))
    @color.setter
    def color(self,value):
        self.viewport.image.putpixel((self.x,self.y),value)
    
    def __complex__(self):
        return ( complex(self.x,self.y)*self.viewport.scale+self.viewport.offset)


In [65]:
from math import log
@dataclass
class ConjMandelbrot:
    max_iter:int
    radio_esc:float
    def __cont__(self,c):
        return self.stability(c)==1
    def stability(self,c,smooth=False,clamp=True):
        value = self.esc_count(c,smooth)/self.max_iter
        return max(0.,min(value,1.)) if clamp else value
    def esc_count(self,c,smooth=False):
        z=0
        for i in range(self.max_iter):
            z=z**2+c
            if abs(z)>self.radio_esc:
                if smooth:
                    return i +1-log(log(abs(z)))/log(2)
                return i
        return self.max_iter

In [66]:
@dataclass
class ViewPort:
    image: Image.Image
    center:complex
    width:float
    @property
    def height(self):
        return self.scale*self.image.height
    @property
    def offset(self):
        return self.center+complex(-self.width, self.height)/2
    @property
    def scale(self):
        return self.width/self.image.width
    
    def __iter__(self):
        for y in range(self.image.height):
            for x in range(self.image.width):
                yield Pixel(self,x,y)


In [117]:
@dataclass
class Pixel:
    viewport:ViewPort
    x:int
    y:int
    
    @property
    def color(self):
        return self.viewport.image.getpixel((self.x,self.y))
    @color.setter
    def color(self,value):
        self.viewport.image.putpixel((self.x,self.y),value)
    
    def __complex__(self):
        return ( complex(self.x,-self.y)*self.viewport.scale+self.viewport.offset)
    

In [118]:
def paint(mandel,viewport,palette,smooth):
    for pixel in viewport:
        stability=mandel.stability(complex(pixel),smooth)
        index=int(min(stability*len(palette),len(palette)-1))
        pixel.color=palette[index % len(palette)]


In [69]:
import matplotlib.cm

def denormalize(palette):
    return [tuple(int(channel * 255) for channel in color) for color in palette]

colormap=matplotlib.cm.get_cmap("twilight").colors
palette=denormalize(colormap)


<ipython-input-69-04f7e07551e7>:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap=matplotlib.cm.get_cmap("twilight").colors


In [70]:
colormap

[[0.8857501584075443, 0.8500092494306783, 0.8879736506427196],
 [0.8837852019553906, 0.8507294054031063, 0.8872322209694989],
 [0.8817223105928579, 0.8512759407765347, 0.8863805692551482],
 [0.8795410528270573, 0.8516567540749572, 0.8854143767924102],
 [0.8772488085896548, 0.8518702833887027, 0.8843412038131143],
 [0.8748534750857597, 0.8519152612302319, 0.8831692696761383],
 [0.8723313408512408, 0.8518016547808089, 0.8818970435500162],
 [0.8697047485350982, 0.8515240300479789, 0.8805388339000336],
 [0.8669601550533358, 0.8510896085314068, 0.8790976697717334],
 [0.86408985081464, 0.8505039116750779, 0.8775792578489263],
 [0.8611024543689985, 0.8497675485700126, 0.8759924292343957],
 [0.8579825924567037, 0.8488893481028184, 0.8743403855344628],
 [0.8547259318925698, 0.8478748812467282, 0.8726282980930582],
 [0.8513371457085719, 0.8467273579611647, 0.8708608165735044],
 [0.8478071070257792, 0.8454546229209523, 0.8690403678369444],
 [0.8441261828674842, 0.8440648271103739, 0.8671697332269

In [71]:
mandel=ConjMandelbrot(max_iter=512,radio_esc=1000)
image=Image.new(mode='RGB',size=(512,512))
viewport=ViewPort(image,center=-0.7435+0.1314j,width=0.002)
paint(mandel,viewport,palette,smooth=True)
image.show()

In [72]:
exterior=[(1,1,1)]*50
interior=[(1,1,1)]*5
areag=[(1-i/44,)*3 for i in range(45)]
palette2=denormalize(exterior+areag+interior)

In [123]:
image2=Image.new(mode='RGB',size=(912,912))
mandel2=ConjMandelbrot(max_iter=20,radio_esc=1000)
viewport2=ViewPort(image2,center=-0.75,width=3.5)
paint(mandel2,viewport2,palette2,smooth=True)
image2.show()

In [80]:
paint(mandel2,viewport,palette2,smooth=True)
image.show()
#image.show()

In [125]:
from scipy.interpolate import interp1d

def make_gradient(colors, interpolation="linear"):
    X = [i / (len(colors) - 1) for i in range(len(colors))]
    Y = [[color[i] for color in colors] for i in range(3)]
    channels = [interp1d(X, y, kind=interpolation) for y in Y]
    return lambda x: [np.clip(channel(x), 0, 1) for channel in channels]


In [143]:
negro=(0,0,0)
azul=(0,0,1)
aq=(0,0.5,0)
mari=(0,0,0.5)
verde=(0,1,0)
colors=[negro,mari,azul,aq,verde,negro]
gradient=make_gradient(colors,interpolation='cubic')

In [144]:
num_colors = 256
palette3 = denormalize([gradient(i / num_colors) for i in range(num_colors)])


In [145]:
image2=Image.new(mode='RGB',size=(912,912))
mandelbrot_set =ConjMandelbrot(max_iter=20, radio_esc=1000)
viewport2=ViewPort(image2,center=-0.75,width=3.5)
paint(mandelbrot_set, viewport2, palette3, smooth=True)
image2.show()